In [1]:
%load_ext autoreload
%autoreload 2

# Register ACP service

First ensure that your ACP tool is up and running by executing the following command in your terminal from the root directory:
```
python search_tool.py
```

In [2]:
import nacos

SERVER_ADDRESSES = "http://localhost:8848"
NAMESPACE = "public"

client = nacos.NacosClient(
    SERVER_ADDRESSES,
    namespace=NAMESPACE,
    # if you need auth:
    # ak="yourAccessKey", sk="yourSecretKey"
)

In [3]:
data_id = "config.nacos"
group = "DEFAULT_GROUP"
print(client.get_config(data_id, group))

# intentionally left blank


In [11]:
import os

SERVICE_NAME = "internet-search-agent"
SERVICE_IP   = os.getenv("SERVICE_IP", "127.0.0.1")
SERVICE_PORT = int(os.getenv("SERVICE_PORT", "8008"))

success = client.add_naming_instance(
    service_name=SERVICE_NAME,
    ip=SERVICE_IP,
    port=SERVICE_PORT, 
    weight=1.0, #A float number for load balancing weight
    ephemeral=False, 
    enable=True,
    healthy=True,
    metadata={"protocol": "acp"}
)
print(f"Registered with Nacos? {success}")

Registered with Nacos? True


Deregister instance

In [ ]:
# SERVICE_NAME = "internet-search-agent"
# SERVICE_IP   = os.getenv("SERVICE_IP", "127.0.0.1")
# SERVICE_PORT = int(os.getenv("SERVICE_PORT", "8008"))

# success = client.remove_naming_instance(
#     service_name=SERVICE_NAME,
#     ip=SERVICE_IP,
#     port=SERVICE_PORT, 
# )
# print(f"De-Registered with Nacos? {success}")

De-Registered with Nacos? True


In [ ]:
from v2.nacos import NacosNamingService, ClientConfig, ListServiceParam

config = ClientConfig(server_addresses=SERVER_ADDRESSES, namespace_id=NAMESPACE)

In [16]:
import nest_asyncio

nest_asyncio.apply()

In [40]:
named_service = await NacosNamingService.create_naming_service(config)

In [ ]:
await named_service.list_services(
    ListServiceParam(namespace_id=NAMESPACE)
)

ServiceList(count=2, services=['forex-tool::1.9.2', 'internet-search-agent'])

In [42]:
client.list_naming_instance(
    "internet-search-agent"
)

{'name': 'DEFAULT_GROUP@@internet-search-agent',
 'groupName': 'DEFAULT_GROUP',
 'clusters': '',
 'cacheMillis': 10000,
 'hosts': [{'instanceId': '127.0.0.1#8008#None#DEFAULT_GROUP@@internet-search-agent',
   'ip': '127.0.0.1',
   'port': 8008,
   'weight': 1.0,
   'healthy': False,
   'enabled': True,
   'ephemeral': False,
   'clusterName': 'None',
   'serviceName': 'DEFAULT_GROUP@@internet-search-agent',
   'metadata': {'protocol': 'acp'},
   'instanceHeartBeatInterval': 5000,
   'instanceHeartBeatTimeOut': 15000,
   'ipDeleteTimeout': 30000}],
 'lastRefTime': 1754401687113,
 'checksum': '',
 'allIps': False,
 'reachProtectionThreshold': False,
 'valid': True}

And thus discoverability was born! Let's ping our "discovered" ACP agent

In [ ]:
from acp_sdk.client import Client
from acp_sdk.models import MessageCompletedEvent, GenericEvent

async with Client(base_url="http://localhost:8008") as client:
    async for event in client.run_stream(
        agent="tavily_search_agent", input="What is a qubit??"
    ):
        match event:
            case MessageCompletedEvent():
                print("\nFinal response:", event.message)
            case GenericEvent():
                print(event.generic.update)

['agent', {'messages': [{'content': 'A qubit, or quantum bit, is the fundamental unit of quantum information in quantum computing. Unlike a classical bit, which can be either 0 or 1, a qubit can exist in a state of 0, 1, or any quantum superposition of these states. This means that a qubit can represent both 0 and 1 simultaneously to some extent, allowing quantum computers to process a vast amount of information more efficiently than classical computers.\n\nKey properties of qubits include:\n\n1. **Superposition**: A qubit can be in a combination of both 0 and 1 states at the same time, which enables quantum computers to perform multiple calculations simultaneously.\n\n2. **Entanglement**: Qubits can be entangled, meaning the state of one qubit is directly related to the state of another, no matter the distance between them. This property is crucial for quantum communication and computing.\n\n3. **Interference**: Quantum algorithms leverage interference to amplify the probabilities of 

# A shadow of MCP-Zero

In [7]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

client = MultiServerMCPClient(
    {
        "mcp_router": {
            # Make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse/",
            "transport": "sse",
        }
    }
)
tools = await client.get_tools()

In [8]:
tools

[StructuredTool(name='search_mcp_server', description='执行任务前首先使用本工具。根据任务描述及关键字搜索mcp server, 制定完成任务的步骤。注意：任务描述及关键字需要同时包含中文和英文。', args_schema={'type': 'object', 'required': ['task_description', 'key_words'], 'properties': {'task_description': {'type': 'string', 'description': '用户中文和英文任务描述，中英文描述各占单独一行。如果任务描述只包含中文，请同时输入英文描述，反之亦然。'}, 'key_words': {'type': 'string', 'description': '用户任务关键字，可以为多个，最多为4个，包含中英文关键字，英文逗号分隔'}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x114077920>),
 StructuredTool(name='add_mcp_server', description='安装指定的mcp server', args_schema={'type': 'object', 'required': ['mcp_server_name'], 'properties': {'mcp_server_name': {'type': 'string', 'description': 'MCP Server名称'}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x117c58e00>),
 StructuredTool(name='use_tool', description='使用某个MCP Server的工具', args_schema={'type': 'object'

Wow the description is in Chinese! 

In [10]:
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

_ = load_dotenv(find_dotenv())
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)
openai_agent = create_react_agent(model=llm, tools=tools)

In [11]:
async for chunk in openai_agent.astream(
    {"messages": [{"role": "user", "content": "What is the exchange rate between USD and EUR?"}]},
    stream_mode="updates"
):
    for value in chunk.items():
        print(value)

('agent', {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZUlLlGFN0mj7Qi1KY3xLnT5G', 'function': {'arguments': '{"task_description":"What is the exchange rate between USD and EUR?","key_words":"exchange rate, USD, EUR"}', 'name': 'search_mcp_server'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 256, 'total_tokens': 292, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C1CaWS0BFoh1CFNWmhaGqwcfXgP2K', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a462cb23-d092-4426-8bfd-abee4aefcae3-0', tool_calls=[{'name': 'search_mcp_server', 'args': {'task_description': 'What is the exchange rate between USD 

The MCP router tool handles the routing for us! We only need to provide it the MCP tool!